# Importe

In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

# Own scripts
from functions import get_start_end_points, generate_histogram2d, gen_spirals, analyse_spiral

# Daten

In [2]:
with open('data.pickle', 'rb') as handle:
    data = pickle.load(handle)
data.head()

,id,run,timestamp,segment,start_time,end_time,force_x,force_y,force_z,pos_x,pos_y,pos_z,rot_x,rot_y,rot_z,result_robot
0,163.0,0.0,2017-05-30 09:21:13.150,11.0,2017-05-30 09:21:05,2017-05-30 09:22:41.607,-0.543045,-2.924596,-7.257000,-257.695704,-265.980105,240.007357,-0.008832,0.034723,179.958414,Success
1,164.0,0.0,2017-05-30 09:21:13.200,11.0,2017-05-30 09:21:05,2017-05-30 09:22:41.607,-0.465969,-3.278856,-7.522000,-257.617629,-265.999590,240.004745,-0.000171,0.019928,179.968161,Success
2,165.0,0.0,2017-05-30 09:21:13.250,11.0,2017-05-30 09:21:05,2017-05-30 09:22:41.607,-0.317475,-3.435834,-7.705999,-256.626803,-266.602650,239.791224,-179.993978,179.989304,0.164766,Success
3,166.0,0.0,2017-05-30 09:21:13.300,11.0,2017-05-30 09:21:05,2017-05-30 09:22:41.607,-0.298383,-3.457754,-7.580999,-254.594017,-267.926090,239.323013,-179.992478,179.999036,0.577883,Success
4,167.0,0.0,2017-05-30 09:21:13.350,11.0,2017-05-30 09:21:05,2017-05-30 09:22:41.607,-0.289898,-3.447854,-7.539000,-251.411919,-269.894497,238.550625,-179.996649,179.988091,1.210269,Success


In [3]:
start_points, end_points, rel_end_points = get_start_end_points(data)

## Relative Startpunkt-Verschiebungen

In [4]:
start_original = (0.,0.)

start_euclidean = (0.025611000000001185, -0.05685299999999903)
start_euclidean_no = (0.020611000000001184, -0.05685299999999903)

start_raster5 = (-0.01861169999999923, -0.1066871999999762)
start_raster10 = (0.006517850000000713, -0.07558529999997746)
start_raster15 = (0.014894366666667352, -0.06521799999997788)
start_raster20 = (0.019082625000000686, -0.028932449999979327)

start_kMeans_c1 = (0.02842986, -0.0424123)
start_kMeans_c2_s1 = (0.01997886, -0.0663818)
start_kMeans_c2_s2 = (0.09221749,  0.13850815)


starts = [start_original, start_euclidean, start_euclidean_no, start_raster5, start_raster10, start_raster15, start_raster20, start_kMeans_c1, start_kMeans_c2_s1, start_kMeans_c2_s2]
names = ["Original", "Euclidean", "Euclidean - no outlier", "Raster 5", "Raster 10", "Raster 15", "Raster 20", "1 Cluster", "2 Cluster Centrum 1", "2 Cluster Centrum 1"]
names_dict = dict(zip(range(len(names)), names))
print(names_dict)

{0: 'Original', 1: 'Euclidean', 2: 'Euclidean - no outlier', 3: 'Raster 5', 4: 'Raster 10', 5: 'Raster 15', 6: 'Raster 20', 7: '1 Cluster', 8: '2 Cluster Centrum 1', 9: '2 Cluster Centrum 1'}


## Schrittgrößen

In [5]:
step_sizes = [0.5, 0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.002, 0.001] #, 0.0005, 0.0002, 0.0001
step_sizes.reverse()

## Funktionen

In [6]:
def spiral_eval(hist, xedge, yedge, i):
    spirals = gen_spirals(hist, xedge, yedge, starts[i][0], starts[i][1])
    return (analyse_spiral(spirals, hist, silent = True)[0], int(i))

In [7]:
def calculate_scores_for_step_size(step_size:float):
    hist, xedge, yedge = generate_histogram2d(rel_end_points, step_size = step_size)

    evaluation = Parallel(n_jobs=-1)(delayed(spiral_eval)(hist, xedge, yedge, i) for i in range(len(starts)))
    #for i in reversed(range(len(starts))):
    #    evaluation.append(spiral_eval(hist, xedge, yedge, i))
    df = pd.DataFrame(evaluation, columns=[step_size, "index"])
    df.set_index("index", inplace=True)
    df.sort_index(inplace=True)
    df.rename(index=names_dict, inplace=True)
    return df

In [8]:
def calculate_scores():
    df = pd.DataFrame(index=names)
    for s in tqdm(step_sizes):
        df = pd.concat([df, calculate_scores_for_step_size(s)], axis=1)
    return df


In [9]:
calculate_scores()

100%|██████████| 9/9 [00:37<00:00,  4.17s/it]


,0.001,0.002,0.005,0.010,0.020,0.050,0.100,0.200,0.500
Original,28513240.0,7106937.0,1123784.0,277321.0,67376.0,10054.0,2532.0,657.0,25.0
Euclidean,25208758.0,6270515.0,994188.0,253615.0,62564.0,9974.0,2862.0,657.0,25.0
Euclidean - no outlier,25186242.0,6267313.0,994076.0,252845.0,62564.0,9938.0,2862.0,657.0,25.0
Raster 5,37178860.0,9167165.0,1454364.0,371825.0,84796.0,12174.0,2862.0,1199.0,25.0
Raster 10,28102766.0,7024427.0,1111228.0,281835.0,70308.0,12174.0,2862.0,657.0,25.0
Raster 15,26186054.0,6573711.0,1038832.0,266637.0,62564.0,9938.0,2862.0,657.0,25.0
Raster 20,24153976.0,6031807.0,959182.0,240423.0,60008.0,9938.0,2532.0,657.0,25.0
1 Cluster,24186720.0,6043373.0,961866.0,241023.0,59888.0,9974.0,2532.0,657.0,25.0
2 Cluster Centrum 1,26226962.0,6546555.0,1036064.0,264907.0,62564.0,9938.0,2862.0,657.0,25.0
2 Cluster Centrum 1,101624478.0,25107235.0,4037914.0,992397.0,239642.0,37552.0,5146.0,1771.0,25.0
